This notebook generates and runs an MCM-CB experiment on IBM Q cloud quantum processors. Running this requires an IBM Q account. This notebook is set up to generate a 4-qubit experiment with a single MCM and 3 idling qubits (as presented in the main text of "Pauli Noise Learning for Mid-Circuit Measurements"). It is readily adaptable to other system sizes (e.g. the 2-qubit example in the SM)

In [1]:
import numpy as np
import pygsti
import pygsti.baseobjs.label as _lbl
import pygsti.modelmembers as mm
from pygsti.circuits import Circuit as _cir
import pygsti.algorithms.randomcircuit as _rc
from pygsti.tools import symplectic as _symp
import itertools
from pygsti.processors import CliffordCompilationRules as CCR

from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Options, Session
from pygsti.extras import devices
from pygsti.extras import ibmq

from mcmrc import *

In [ ]:
#IBMQ.load_account()
device_name = 'ibm_brisbane' #put the device name you want to use
service = QiskitRuntimeService()
backend = service.backend(device_name)

In [146]:
qubit_labels = ('Q40','Q41','Q42','Q43')
n = len(qubit_labels)
availability = {'Gcphase':[(f'Q{i}',f'Q{i+1}') for i in range(40,43)]}
gate_names = [f'Gc{i}' for i in range(24)]+['Gcphase']
pspec = pygsti.processors.QubitProcessorSpec(n, gate_names, availability=availability, qubit_labels=qubit_labels) 

In [182]:
model = pygsti.models.modelconstruction.create_crosstalk_free_model(pspec)

In [147]:
depths = [0,2,4,8,12]
num_randomizations = 30
num_meas = 1
num_unmeas = 3
measured_qs = qubit_labels[:num_meas]
unmeasured_qs = [q for q in qubit_labels if q not in measured_qs]

cycle = pygsti.circuits.Circuit([[pygsti.baseobjs.label.Label('Iz',('Q40'))],[pygsti.baseobjs.label.Label('Gc0',f'{i}') for i in qubit_labels]])

In [148]:
compilations = {'absolute': CCR.create_standard(pspec, 'absolute', ('paulis', '1Qcliffords'), verbosity=0),            
                'paulieq': CCR.create_standard(pspec, 'paulieq', ('1Qcliffords', 'allcnots'), verbosity=0)}

In [149]:
unmeas_paulis = itertools.product(['I','X','Y','Z'], repeat=num_unmeas)
required_paulis = [('Z',)+p for p in unmeas_paulis]

In [150]:
edesigns = {}
cs_by_pauli, signs_by_pauli, tbs_by_pauli = generate_circuits_rc(cycle, depths, required_paulis, compilations, num_randomizations, qubit_labels)
for pauli, clists in cs_by_pauli.items():
    print(pauli)
    edesigns[pauli] = pygsti.protocols.ByDepthDesign(depths, clists)
ced = pygsti.protocols.CombinedExperimentDesign(edesigns)

('Z', 'I', 'I', 'I')
('Z', 'I', 'I', 'X')
('Z', 'I', 'I', 'Y')
('Z', 'I', 'I', 'Z')
('Z', 'I', 'X', 'I')
('Z', 'I', 'X', 'X')
('Z', 'I', 'X', 'Y')
('Z', 'I', 'X', 'Z')
('Z', 'I', 'Y', 'I')
('Z', 'I', 'Y', 'X')
('Z', 'I', 'Y', 'Y')
('Z', 'I', 'Y', 'Z')
('Z', 'I', 'Z', 'I')
('Z', 'I', 'Z', 'X')
('Z', 'I', 'Z', 'Y')
('Z', 'I', 'Z', 'Z')
('Z', 'X', 'I', 'I')
('Z', 'X', 'I', 'X')
('Z', 'X', 'I', 'Y')
('Z', 'X', 'I', 'Z')
('Z', 'X', 'X', 'I')
('Z', 'X', 'X', 'X')
('Z', 'X', 'X', 'Y')
('Z', 'X', 'X', 'Z')
('Z', 'X', 'Y', 'I')
('Z', 'X', 'Y', 'X')
('Z', 'X', 'Y', 'Y')
('Z', 'X', 'Y', 'Z')
('Z', 'X', 'Z', 'I')
('Z', 'X', 'Z', 'X')
('Z', 'X', 'Z', 'Y')
('Z', 'X', 'Z', 'Z')
('Z', 'Y', 'I', 'I')
('Z', 'Y', 'I', 'X')
('Z', 'Y', 'I', 'Y')
('Z', 'Y', 'I', 'Z')
('Z', 'Y', 'X', 'I')
('Z', 'Y', 'X', 'X')
('Z', 'Y', 'X', 'Y')
('Z', 'Y', 'X', 'Z')
('Z', 'Y', 'Y', 'I')
('Z', 'Y', 'Y', 'X')
('Z', 'Y', 'Y', 'Y')
('Z', 'Y', 'Y', 'Z')
('Z', 'Y', 'Z', 'I')
('Z', 'Y', 'Z', 'X')
('Z', 'Y', 'Z', 'Y')
('Z', 'Y', 'Z

In [152]:
exp_dict = {'cs':cs_by_pauli, 'tbs':tbs_by_pauli, 'signs':signs_by_pauli}

In [156]:
qubit_labels = tuple(f'Q{i}' for i in range(44))
n = len(qubit_labels)
availability = {'Gcphase':[(f'Q{i}',f'Q{i+1}') for i in range(0,44)]}
gate_names = [f'Gc{i}' for i in range(24)]+['Gcphase']#['Gc16', 'Gzr']+['Gcphase']
big_pspec = pygsti.processors.QubitProcessorSpec(n, gate_names, availability=availability, qubit_labels=qubit_labels) 

In [157]:
#let's hope pyGSTi knows how to do the qasm decompositions
exp = ibmq.IBMQExperiment(ced, big_pspec, circuits_per_batch = 100, num_shots = 5000)

Constructing job for circuit batch 1 of 96
Constructing job for circuit batch 2 of 96
Constructing job for circuit batch 3 of 96
Constructing job for circuit batch 4 of 96
Constructing job for circuit batch 5 of 96
Constructing job for circuit batch 6 of 96
Constructing job for circuit batch 7 of 96
Constructing job for circuit batch 8 of 96
Constructing job for circuit batch 9 of 96
Constructing job for circuit batch 10 of 96
Constructing job for circuit batch 11 of 96
Constructing job for circuit batch 12 of 96
Constructing job for circuit batch 13 of 96
Constructing job for circuit batch 14 of 96
Constructing job for circuit batch 15 of 96
Constructing job for circuit batch 16 of 96
Constructing job for circuit batch 17 of 96
Constructing job for circuit batch 18 of 96
Constructing job for circuit batch 19 of 96
Constructing job for circuit batch 20 of 96
Constructing job for circuit batch 21 of 96
Constructing job for circuit batch 22 of 96
Constructing job for circuit batch 23 of 

In [158]:
import qiskit
from qiskit.circuit.library import XGate
from qiskit.transpiler import PassManager, InstructionDurations
from qiskit.transpiler.passes import ALAPScheduleAnalysis, PadDynamicalDecoupling
from qiskit.visualization import timeline_drawer


def insert_dd(circ, backend):
    durations = qiskit.transpiler.InstructionDurations.from_backend(backend)
# balanced X-X sequence on all qubits
    dd_sequence = [XGate(), XGate()]
    pm = PassManager([ALAPScheduleAnalysis(durations),
                  PadDynamicalDecoupling(durations, dd_sequence)])
    circ_dd = pm.run(circ)
    return circ_dd

def add_dd_to_exp(exp, backend):
    for i, batch in enumerate(exp['qiskit_QuantumCircuits']):
        circuits = []
        for qiskit_circ in batch:
            circ_dd = insert_dd(qiskit_circ, backend)
            circuits.append(circ_dd)
        exp['qiskit_QuantumCircuits'][i] = circuits
    return exp

In [159]:
exp = add_dd_to_exp(exp,backend)

In [160]:
with backend.open_session() as session:
    exp.submit(backend)

/tmp/ipykernel_73903/4020636989.py:1: DeprecationWarning: backend.run() and related sessions methods are deprecated  as of qiskit-ibm-runtime 0.23 and will be removed no sooner than 6 months after the release date. More details can be found in the primitives migration guide https://docs.quantum.ibm.com/api/migration-guides/qiskit-runtime.
  with backend.open_session() as session:


Submitting batch 1
  - Job ID is cs4m1sqkfpw00080m6tg
  - Failed to get queue position for batch 1
Submitting batch 2
  - Job ID is cs4m1tf965y000857xc0
  - Failed to get queue position for batch 2
Submitting batch 3
  - Job ID is cs4m1tzkfpw00080m6v0
  - Failed to get queue position for batch 3
Submitting batch 4
  - Job ID is cs4m1vf75q40008tvtw0
  - Failed to get queue position for batch 4
Submitting batch 5
  - Job ID is cs4m1w74amg00087p9ng
  - Failed to get queue position for batch 5
Submitting batch 6
  - Job ID is cs4m1wq4amg00087p9p0
  - Failed to get queue position for batch 6
Submitting batch 7
  - Job ID is cs4m1x7yhpyg008arbag
  - Queue position is 1
Submitting batch 8
  - Job ID is cs4m1xqyhpyg008arbb0
  - Queue position is 2
Submitting batch 9
  - Job ID is cs4m1y775q40008tvtwg
  - Queue position is 3
Submitting batch 10
  - Job ID is cs4m1yqkfpw00080m6vg
  - Queue position is 4
Submitting batch 11
  - Job ID is cs4m1z7kfpw00080m6w0
  - Queue position is 5
Submitting bat

In [92]:
exp.monitor()

Batch 1: JobStatus.DONE
Batch 2: JobStatus.DONE
Batch 3: JobStatus.DONE
Batch 4: JobStatus.DONE
Batch 5: JobStatus.DONE
Batch 6: JobStatus.DONE
Batch 7: JobStatus.DONE
Batch 8: JobStatus.DONE
Batch 9: JobStatus.DONE
Batch 10: JobStatus.DONE
Batch 11: JobStatus.DONE
Batch 12: JobStatus.DONE
Batch 13: JobStatus.DONE
Batch 14: JobStatus.DONE
Batch 15: JobStatus.DONE
Batch 16: JobStatus.DONE
Batch 17: JobStatus.DONE
Batch 18: JobStatus.DONE
Batch 19: JobStatus.DONE
Batch 20: JobStatus.DONE
Batch 21: JobStatus.DONE
Batch 22: JobStatus.DONE
Batch 23: JobStatus.DONE
Batch 24: JobStatus.DONE
Batch 25: JobStatus.DONE
Batch 26: JobStatus.DONE
Batch 27: JobStatus.DONE
Batch 28: JobStatus.DONE
Batch 29: JobStatus.DONE
Batch 30: JobStatus.DONE
Batch 31: JobStatus.DONE
Batch 32: JobStatus.DONE
Batch 33: JobStatus.DONE
Batch 34: JobStatus.DONE
Batch 35: JobStatus.DONE
Batch 36: JobStatus.DONE
Batch 37: JobStatus.DONE
Batch 38: JobStatus.DONE
Batch 39: JobStatus.DONE
Batch 40: JobStatus.DONE
Batch 41:

In [161]:
def to_labeled_counts(input_dict, ordered_target_indices, num_qubits_in_pspec): 
    outcome_labels = []
    counts_data = []
    for bitstring, count in input_dict.items():
        new_label = []
        term_string = ''
        term_bits = bitstring[:num_qubits_in_pspec][::-1]
        mid_bits = bitstring[num_qubits_in_pspec:][::-1]
        for index in ordered_target_indices:
            term_string += term_bits[index]
        for bit in mid_bits:
            new_label.append('p'+bit)
        new_label.append(term_string)
        outcome_labels.append(tuple(new_label))
        counts_data.append(count)
    return outcome_labels, counts_data

In [162]:
#exp.retrieve_results()
exp['batch_result_object'] = []
#get results from backend jobs and add to dict
ds = pygsti.data.DataSet()
for exp_idx, qjob in enumerate(exp['qjob']):
    print("Querying IBMQ for results objects for batch {}...".format(exp_idx + 1))
    try:
        batch_result = qjob.result()
        exp['batch_result_object'].append(batch_result)
        #exp_dict['batch_data'] = []
        num_qubits_in_pspec = exp['pspec'].num_qubits
        for i, circ in enumerate(exp['pygsti_circuits'][exp_idx]): 
            ordered_target_indices = [exp['pspec'].qubit_labels.index(q) for q in circ.line_labels]
            #assumes qubit labeling of the form 'Q0' not '0' 
            labeled_counts = to_labeled_counts(batch_result.get_counts(i), ordered_target_indices, num_qubits_in_pspec)
            outcome_labels = labeled_counts[0]
            counts_data = labeled_counts[1]
            ds.add_count_list(circ, outcome_labels, counts_data)
    except:
        print('failed to retrieve batch')
exp['data'] = pygsti.protocols.ProtocolData(exp['edesign'], ds)
print('Finished retrieving results!') #Changed this 

Querying IBMQ for results objects for batch 1...
Querying IBMQ for results objects for batch 2...
Querying IBMQ for results objects for batch 3...
Querying IBMQ for results objects for batch 4...
Querying IBMQ for results objects for batch 5...
Querying IBMQ for results objects for batch 6...
Querying IBMQ for results objects for batch 7...
Querying IBMQ for results objects for batch 8...
Querying IBMQ for results objects for batch 9...
Querying IBMQ for results objects for batch 10...
Querying IBMQ for results objects for batch 11...
Querying IBMQ for results objects for batch 12...
Querying IBMQ for results objects for batch 13...
Querying IBMQ for results objects for batch 14...
Querying IBMQ for results objects for batch 15...
Querying IBMQ for results objects for batch 16...
Querying IBMQ for results objects for batch 17...
Querying IBMQ for results objects for batch 18...
Querying IBMQ for results objects for batch 19...
Querying IBMQ for results objects for batch 20...
Querying 

In [163]:
pd = exp['data']
pd.write('05_18_test_mcmcb_ibm_osaka')